# 🎯 Génération de Formulaires avec Llama 3.2 3B

Ce notebook permet de charger votre modèle Llama 3.2 3B entraîné et de générer des structures de formulaires JSON.

**Configuration:**
- Runtime: Python 3
- Hardware accelerator: GPU (optionnel mais recommandé)

**Prérequis:**
- Avoir entraîné le modèle avec `train_colab.ipynb`
- Le modèle doit être disponible sur Google Drive

## 📋 Étape 1: Installation des dépendances

In [ ]:
# Installation des bibliothèques
!pip install -q -U transformers accelerate peft sentencepiece

print("✅ Installation terminée!")

In [ ]:
# Vérifier le matériel disponible
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("CPU mode - La génération sera plus lente")

## 💾 Étape 2: Monter Google Drive

Pour accéder au modèle sauvegardé.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Vérifier que le modèle existe
import os

model_path = "/content/drive/MyDrive/llama3-form-generator"

if os.path.exists(model_path):
    print(f"✅ Modèle trouvé: {model_path}")
    # Lister les fichiers
    files = os.listdir(model_path)
    print(f"\nFichiers: {', '.join(files[:5])}...")
else:
    print(f"❌ Modèle non trouvé à: {model_path}")
    print("\nAssurez-vous d'avoir:")
    print("1. Entraîné le modèle avec train_colab.ipynb")
    print("2. Sauvegardé le modèle sur Google Drive")

## 🔄 Étape 3: Charger le modèle

Chargement du modèle entraîné.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Chemin du modèle
model_path = "/content/drive/MyDrive/llama3-form-generator"

print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("📥 Chargement du modèle...")
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

model.eval()

print(f"\n✅ Modèle chargé sur {device}!")

## 🛠️ Étape 4: Définir la fonction de génération

In [ ]:
import json

def generate_form(description, max_new_tokens=1024, temperature=0.7, top_p=0.9, verbose=True):
    """
    Génère une structure de formulaire à partir d'une description.

    Args:
        description: Description du formulaire
        max_new_tokens: Nombre maximum de tokens à générer
        temperature: Température (0.0-1.0)
        top_p: Top-p sampling
        verbose: Afficher les détails

    Returns:
        dict: Structure de formulaire JSON
    """
    if verbose:
        print(f"📝 Description: {description}\n")

    # Format de prompt pour Llama 3.2 Instruct
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Tu es un assistant spécialisé dans la génération de structures de formulaires JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>

{description}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    if verbose:
        print("🔄 Génération en cours...")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraire la réponse
    if "assistant" in generated_text:
        response = generated_text.split("assistant")[1].strip()
    else:
        response = generated_text.strip()

    response = response.replace("<|eot_id|>", "").strip()

    # Parser le JSON
    try:
        form_structure = json.loads(response)
        if verbose:
            print("✅ Génération réussie!\n")
        return form_structure
    except json.JSONDecodeError as e:
        if verbose:
            print(f"⚠️ Erreur de parsing JSON: {e}")
            print(f"Réponse brute: {response[:200]}...\n")
        return {"raw_output": response, "error": str(e)}

print("✅ Fonction de génération définie!")

## 🎨 Étape 5: Exemples de génération

In [ ]:
# Exemple 1: Formulaire d'inscription
print("=" * 60)
print("EXEMPLE 1: Formulaire d'inscription")
print("=" * 60)

form1 = generate_form(
    "Crée un formulaire d'inscription avec nom, prénom, email, téléphone et adresse"
)

print("📋 Résultat:")
print(json.dumps(form1, indent=2, ensure_ascii=False))

In [ ]:
# Exemple 2: Formulaire de contact
print("=" * 60)
print("EXEMPLE 2: Formulaire de contact")
print("=" * 60)

form2 = generate_form(
    "Crée un formulaire de contact avec nom, email, sujet et message"
)

print("📋 Résultat:")
print(json.dumps(form2, indent=2, ensure_ascii=False))

In [ ]:
# Exemple 3: Formulaire de commande
print("=" * 60)
print("EXEMPLE 3: Formulaire de commande")
print("=" * 60)

form3 = generate_form(
    "Crée un formulaire de commande avec produit, quantité, prix et adresse de livraison"
)

print("📋 Résultat:")
print(json.dumps(form3, indent=2, ensure_ascii=False))

## ✏️ Étape 6: Génération personnalisée

Modifiez la description ci-dessous pour générer votre propre formulaire.

In [ ]:
# 🎯 Personnalisez votre description ici
custom_description = "Crée un formulaire d'inscription à un événement avec nom, prénom, email, entreprise et commentaires"

print("=" * 60)
print("GÉNÉRATION PERSONNALISÉE")
print("=" * 60)

custom_form = generate_form(
    custom_description,
    temperature=0.7,  # Ajustez la créativité (0.0-1.0)
    top_p=0.9,        # Ajustez la diversité (0.0-1.0)
)

print("📋 Résultat:")
print(json.dumps(custom_form, indent=2, ensure_ascii=False))

## 📦 Étape 7: Génération en batch

Générez plusieurs formulaires d'un coup.

In [ ]:
# Liste de descriptions
descriptions = [
    "Crée un formulaire de feedback avec note de 1 à 5 et commentaire",
    "Crée un formulaire de candidature avec nom, CV et lettre de motivation",
    "Crée un formulaire de réservation avec date, heure et nombre de personnes",
]

print("=" * 60)
print("GÉNÉRATION EN BATCH")
print("=" * 60)

batch_forms = []
for i, desc in enumerate(descriptions, 1):
    print(f"\n📝 Formulaire {i}/{len(descriptions)}")
    form = generate_form(desc, verbose=False)
    batch_forms.append(form)
    print(f"✅ Généré!")

print("\n" + "=" * 60)
print("RÉSULTATS")
print("=" * 60)

for i, form in enumerate(batch_forms, 1):
    print(f"\n📋 Formulaire {i}:")
    print(json.dumps(form, indent=2, ensure_ascii=False))
    print()

## 💾 Étape 8: Exporter les résultats

Sauvegardez vos formulaires générés.

In [ ]:
# Sauvegarder dans un fichier JSON
output_file = "generated_forms.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(batch_forms, f, indent=2, ensure_ascii=False)

print(f"✅ Résultats sauvegardés dans: {output_file}")

# Télécharger le fichier
from google.colab import files
files.download(output_file)

print("📥 Téléchargement lancé!")

In [ ]:
# Sauvegarder sur Google Drive
import shutil

drive_output = "/content/drive/MyDrive/generated_forms.json"
shutil.copy(output_file, drive_output)

print(f"✅ Résultats sauvegardés sur Drive: {drive_output}")

## 🎮 Étape 9: Mode interactif

Générez des formulaires interactivement.

In [ ]:
# Mode interactif
while True:
    print("\n" + "=" * 60)
    description = input("📝 Décrivez le formulaire (ou 'quit' pour quitter): ")

    if description.lower() in ['quit', 'q', 'exit']:
        print("👋 Au revoir!")
        break

    if not description.strip():
        print("⚠️ Description vide!")
        continue

    form = generate_form(description)
    print("\n📋 Résultat:")
    print(json.dumps(form, indent=2, ensure_ascii=False))

## 💡 Conseils et astuces

### Paramètres de génération:
- **temperature** (0.0-1.0): Contrôle la créativité
  - Basse (0.1-0.3): Résultats plus déterministes
  - Moyenne (0.5-0.7): Bon équilibre
  - Haute (0.8-1.0): Plus créatif mais moins prévisible

- **top_p** (0.0-1.0): Contrôle la diversité
  - Valeur recommandée: 0.9

- **max_new_tokens**: Nombre de tokens à générer
  - Formulaire simple: 512
  - Formulaire complexe: 1024-2048

### Conseils pour de meilleurs résultats:
1. Soyez spécifique dans vos descriptions
2. Listez tous les champs nécessaires
3. Mentionnez les types de champs si nécessaire (select, checkbox, etc.)
4. Indiquez les champs obligatoires si pertinent
5. Ajustez la température selon vos besoins

### Exemples de descriptions efficaces:
- ✅ "Crée un formulaire d'inscription avec nom (texte obligatoire), email (obligatoire), téléphone (optionnel) et pays (menu déroulant)"
- ✅ "Crée un formulaire de commande avec choix de produit, quantité, mode de livraison (express/standard) et adresse complète"
- ❌ "Fais un formulaire" (trop vague)

### Dépannage:
- Si le JSON est invalide, essayez de diminuer la température
- Si les résultats sont trop répétitifs, augmentez la température
- Si le modèle est lent, vérifiez que vous utilisez un GPU
- En cas d'erreur de mémoire, réduisez max_new_tokens